In [41]:
import arcpy
import glob
from tqdm.notebook import tqdm
from arcpy.sa import *

# ArcGIS setup
arcpy.env.overwriteOutput = True
arcpy.env.extent = "MAXOF"

# set workspace path
workspace = r"E:/fires/fires.gdb/"
arcpy.env.workspace = workspace

# root folder
root_path = r"E:/fires/"

# contains MTBS data 1984-2018 as rasters
data_path = r"E:/fires/mtbs/"

# convert to binary

we only care about the most severe fires (rated as 4 in the rasters). We thus mask out all values that are not zero. Unfortunately, this process is extremely inefficient even on high-end hardware. Classic ArcPy.

In [29]:
# get a list of files to process
raw_mtbs_list = glob.glob(data_path + "*.tif")

# check out the spatial analyst tools
arcpy.CheckOutExtension("Spatial")

# loop through all the files to process
for mtbs_raster in tqdm(raw_mtbs_list):
    
    # set as a raster
    mtbs_raster = arcpy.Raster(mtbs_raster)
    
    # convert to a binary mask of only "4" (high intensity fires)
    binary_raster = arcpy.sa.SetNull(mtbs_raster, 1, "VALUE < 4")
    
    # get a cleaner savename
    save_name = arcpy.Describe(mtbs_raster).basename + "_binary"
    
    # save the binary raster
    binary_raster.save(save_name)
    
# check in the spatial analyst tools
arcpy.CheckInExtension("Spatial")

# combine into one map via raster calculator manually

this produces a map where each pixel value represents the number of times that cell has been severely burned.

# use zonal statistics to 'downsample' to other data

the MTBS data is much finer than the other data in the models and thus needs to be downsampled to match.

In [ ]:
# get the binary raster
value_raster = r"E:/fires/fires.gdb/binary_sum"

# get any of the other data maps to downsample to
zone_raster = workspace + "cal_zero"

# create a function to save a few lines of coding
def zonal_stats(stat_type, save_name):

    # calculate thdown_sample_cell_sizeple_cell_sizesample_cell_sizet value
    stat_raster = ZonalStatistics(zone_raster, "VALUE", value_raster, stat_type, "DATA")

    # save the results
    stat_raster.save(save_name)

# generate and save all the statistics we could want
zonal_stats("MEDIAN", "binary_sum_median")
zonal_stats("MEAN", "binary_sum_mean")
zonal_stats("MODE", "binary_sum_mode")
zonal_stats("MAX", "binary_sum_max")